In [37]:
from nltk import word_tokenize
import re
from transformers import AutoModel, AutoTokenizer
from transformers import CamembertModel, CamembertTokenizer
from transformers import AutoModelForSequenceClassification, CamembertForMaskedLM, AutoTokenizer, AutoConfig
import torch
import torch.nn.functional as F
import numpy as np

### Méthode 1 : match par embedding des ngrams

L'idée c'est de découper le corps de texte du CV en plusieurs morceaux longs chacun de n mots avec n a definir. Ces sous-sequences sont vectorisées puis comparées a la vectorisation du bout de texte à matcher

Ici, le modele utilisé pour faire de l'embedding est un transformateur de la langue francaise, camemBERT, disponible via le lien https://huggingface.co/camembert-base


In [46]:
#tokenization des phrases, retrait des caracteres qui ne sont pas alphabetiques latin/qui peuvent figurer dans une phrase
def tokenize_st(st : str):
    return [word.lower() for word in word_tokenize(re.sub(r"[^a-zA-z\-éèêàáàâôöîïûüœç]", " ", st))]

#méthodes créant les ngrams
def ngram(words : list, n : int):
    ngrams = []
    if len(words)<n :
        return ngrams
    for i in range(len(words)-n+1):
        ngrams += [words[i:i+n]]
    return ngrams

def generate_ngram_list(st:str, beg=1, end = 5):
    words = tokenize_st(st)
    ngrams = []
    for i in range(beg, end+1) :
        igram = ngram(words, i)
        if len(igram)>0:
            ngrams += [igram]
    return ngrams 

def ngram_to_str(ngrams:list):
    l = []
    it = 0
    for rep in ngrams :
        l+= [[]]
        for sent in rep :
            l[it] += [" ".join(sent)]
        it +=1
    return l

texte = "5 ans d'études pour obtenir un master ou un master of science (Msc) spécialisés en mathématiques appliquées, informatique décisionnelle , data science, statistiques, etc. ou un diplôme d'ingénieur avec double compétence en mathématiques et informatique ou spécialisation big data."
truc_a_trouver = ["bac + 5 master en mathématiques"]

text_ngram = ngram_to_str(generate_ngram_list(texte, 8, 10))
text_ngram

[['ans d études pour obtenir un master ou',
  'd études pour obtenir un master ou un',
  'études pour obtenir un master ou un master',
  'pour obtenir un master ou un master of',
  'obtenir un master ou un master of science',
  'un master ou un master of science msc',
  'master ou un master of science msc spécialisés',
  'ou un master of science msc spécialisés en',
  'un master of science msc spécialisés en mathématiques',
  'master of science msc spécialisés en mathématiques appliquées',
  'of science msc spécialisés en mathématiques appliquées informatique',
  'science msc spécialisés en mathématiques appliquées informatique décisionnelle',
  'msc spécialisés en mathématiques appliquées informatique décisionnelle data',
  'spécialisés en mathématiques appliquées informatique décisionnelle data science',
  'en mathématiques appliquées informatique décisionnelle data science statistiques',
  'mathématiques appliquées informatique décisionnelle data science statistiques etc',
  'appliq

In [47]:
#tokenizer
tokenizer = AutoTokenizer.from_pretrained('camembert-base')
#modele tranformateur camembert
camembert = CamembertForMaskedLM.from_pretrained('camembert-base')

#
def average_embeddings(embeddings, attention_mask):
    return (attention_mask[..., None] * embeddings).mean(1)

def embbed_sentences(camembert, tokenizer, sentences) :
    #on tokenize grace au tokenizer de camembert
    tokenizer_output = tokenizer(sentences,padding="max_length",truncation=True,return_tensors="pt")
    #on fait passer la phrase tokenizee par le modele et on recupere les poids de la derniere 
    with torch.no_grad():
        model_output = camembert(**tokenizer_output, output_hidden_states=True)
    token_embeddings = model_output.hidden_states[-1]
    return average_embeddings(token_embeddings, tokenizer_output.attention_mask)


In [48]:
sentence_to_be_matched = embbed_sentences(camembert, tokenizer, truc_a_trouver)
doc_embedded = embbed_sentences(camembert, tokenizer, text_ngram[0])

In [49]:
def compute_vector_similarity(sentence, doc):
    return [float(F.cosine_similarity(sentence, sent_doc)[0]) for sent_doc in doc]

similar_vect = compute_vector_similarity(sentence_to_be_matched, doc_embedded)
print("sentence to be matched :", truc_a_trouver[0])
print("sentence matched : ",text_ngram[0][np.argmax(similar_vect)])

sentence to be matched : bac + 5 master en mathématiques
sentence matched :  un master of science msc spécialisés en mathématiques


### Méthode 2 : match parfait

Il s'agit tout simplement de verifier si l'intitulé exact de ce qui est recherché figure dans le CV

In [50]:
corps = "5 ans d'études pour obtenir un master ou un master of science (Msc) spécialisés en mathématiques appliquées, informatique décisionnelle , data science, statistiques, etc. ou un diplôme d'ingénieur avec double compétence en mathématiques et informatique ou spécialisation big data."

truc_a_trouver = "mathématiques appliquées"

print(truc_a_trouver in corps, "truc sans rapport" in corps)

True False
